# Overview

This notebook presents a brief tutorial on using [spdlog](https://github.com/gabime/spdlog) as the logging engine for a c++ application.

## Table of content

0. Quick preview
1. Introduction
2. Setting up `spdlog`
3. `poco` and `spdlog`
4. Advanced `spdlog` features
5. Where to go from here

Here we are using `xeus-cling` to add a `c++` kernel for the Jupyter notebook.
`spdlog` is not depeneding on either `jupyter` or `xeus-cling` and can be used in any C++ application. 

In [1]:
#pragma cling add_include_path("/home/8cz/micromamba/envs/spdlog_tutorial/include")
#pragma cling add_library_path("/home/8cz/micromamba/envs/spdlog_tutorial/lib")

In [2]:
#pragma cling load("spdlog")

In [15]:
#pragma cling load("poco")

## 0. Quick preview

In [3]:
#include <spdlog/spdlog.h>

`spdlog` has support for all common logging levels and provide easy access to the default sink.

In [4]:
spdlog::info("***Welcome to spdlog!***");

In [5]:
spdlog::error("Some error message with arg: {}", 1);

In [6]:
spdlog::warn("Easy padding in numbers like {:08d}", 12);

In [7]:
spdlog::critical("Support for int: {0:d};  hex: {0:x};  oct: {0:o}; bin: {0:b}", 42);

it also supports c++ built-in formatting syntax

In [8]:
spdlog::info("Support for floats {:03.2f}", 1.23456);
spdlog::info("Positional args are {1} {0}..", "too", "supported");
spdlog::info("{:<30}", "left aligned");

can also change the debug level on-the-fly

In [9]:
spdlog::set_level(spdlog::level::debug);
spdlog::debug("This DEBUG message will be displayed..");

Create stdout/stderr logger object

In [10]:
#include <spdlog/sinks/stdout_color_sinks.h>

auto console = spdlog::stdout_color_mt("console");
auto err_logger = spdlog::stderr_color_mt("stderr");
spdlog::get("console")->info("loggers can be retrieved from a global registry using the spdlog::get(logger_name)");

## 1. Introduction

1.1. Why Switch to spdlog?

spdlog is a modern C++ logging library that offers high-performance and is feature-rich.
Here are some reasons why we should consider switching to spdlog:

- **Performance**: spdlog is known for its speed and **low** overhead, particularly for asynchronous logging.
- **Flexibility**: It provides a variety of logging targets, known as sinks, such as console, file, rotating file, etc.
- **User-friendly**: Its syntax is intuitive and easily configurable.
- **Actively Maintained**: spdlog has an active community, ensuring regular updates and addressing issues promptly.

1.2. Key Differences between Poco and spdlog

While both Poco and spdlog provide logging capabilities, the design principles are different.
Here are a few examples:

- **Initialization**:
  - Poco: Requires more verbose setup for logging channels.
  - spdlog: Simplified setup with easy-to-use factory functions.
  
- **Performance**:
  - Poco: Good performance, but can be slower in some scenarios.
  - spdlog: Designed for high-performance scenarios, especially with its asynchronous mode.
  
- **Syntax**:
  - Poco: Uses traditional logging syntax with different methods for levels.
  - spdlog: Uses a unified syntax with **format strings**.
  
- **Extensibility**:
  - Poco: Provides extensive features beyond logging, making it heavier but versatile.
  - spdlog: Focuses primarily on logging (single purpose).


## 2. Setting up `spdlog`

`spdlog` is a **header only** library, so there is no need to build it.
As demonstrated in this tutorial, it is available from `conda-forge` and can be easily added as a dependency to your project.

## 3. Poco and spdlog

3.1 Simple logging

`poco`

In [ ]:
%%executable poco_simple_logger

#include <Poco/Logger.h>
#include <Poco/ConsoleChannel.h>

int main() {
    // Setting up Poco logger
    Poco::Logger& logger = Poco::Logger::get("ExampleLogger"); // Create or get logger
    logger.setChannel(new Poco::ConsoleChannel); // Set output channel
    logger.setLevel(Poco::Message::PRIO_DEBUG);  // Set log level

    // Simple loop
    for (int i = 0; i < 5; i++) {
        logger.debug("Poco Loop iteration: " + std::to_string(i));
    }

    return 0;
}

In [ ]:
!./poco_simple_logger

`spdlog`

In [ ]:
%%executable spdlog_simple_logger

#include <spdlog/spdlog.h>
#include <spdlog/sinks/stdout_color_sinks.h>

int main() {
    // Setting up spdlog logger
    auto console = spdlog::stdout_color_mt("ExampleLogger"); // Create or get logger
    spdlog::set_level(spdlog::level::debug); // Set log level

    // Simple loop
    for (int i = 0; i < 5; i++) {
        spdlog::debug("spdlog Loop iteration: {}", i);
    }

    return 0;
}

In [ ]:
!./spdlog_simple_logger

**_Comparison_**:

- Initialization:
  - Poco: Requires setting up the logger, channel, and level separately.
  - spdlog: Offers a more streamlined setup, particularly with predefined sinks like stdout_color_mt.
- Syntax:
  - Poco: Uses string concatenation for logging messages.
  - spdlog: Uses a format string syntax (similar to Python's f-strings or printf), making it concise and more readable.
- Performance:
  - While both will handle this simple scenario with ease, in more extensive logging scenarios, spdlog might offer better performance, especially with its asynchronous logging capabilities.

3.2 With Namespace

`poco`

In [ ]:
%%executable poco_multiple_loggers

#include <Poco/Logger.h>
#include <Poco/ConsoleChannel.h>
#include <string>

namespace namespace_a {
    void logMessage() {
        Poco::Logger& logger = Poco::Logger::get("namespace_a");
        logger.debug("This is a debug message from namespace_a");
    }
}

namespace namespace_b {
    void logMessage() {
        Poco::Logger& logger = Poco::Logger::get("namespace_b");
        logger.debug("This is a debug message from namespace_b");
    }
}

int main() {
    Poco::Logger::root().setChannel(new Poco::ConsoleChannel);
    Poco::Logger::root().setLevel(Poco::Message::PRIO_DEBUG);
    
    namespace_a::logMessage();
    namespace_b::logMessage();

    return 0;
}

In [ ]:
!./poco_multiple_loggers

`spdlog`

In [ ]:
%%executable spdlog_multiple_loggers

#include <spdlog/spdlog.h>
#include <spdlog/sinks/stdout_color_sinks.h>

namespace namespace_a {
    void logMessage() {
        auto logger = spdlog::get("namespace_a");
        if (!logger) {
            logger = spdlog::stdout_color_mt("namespace_a");
        }
        logger->debug("This is a debug message from namespace_a");
    }
}

namespace namespace_b {
    void logMessage() {
        auto logger = spdlog::get("namespace_b");
        if (!logger) {
            logger = spdlog::stdout_color_mt("namespace_b");
        }
        logger->debug("This is a debug message from namespace_b");
    }
}

int main() {
    spdlog::set_level(spdlog::level::debug);
    
    namespace_a::logMessage();
    namespace_b::logMessage();

    return 0;
}


In [ ]:
!./spdlog_multiple_loggers

**_Comparison_**:

- Logger Creation:
  - Poco: Uses the Poco::Logger::get method to either retrieve an existing logger or create a new one. Logger creation is lazy (i.e., it's created when first called).
  - spdlog: Uses the spdlog::get method to retrieve a logger, and if it doesn't exist, it creates one using the spdlog::stdout_color_mt method. This approach ensures that the logger is created only once.
- Namespacing:
  For both Poco and spdlog, the logger name acts as an implicit namespace. The name can be used to differentiate loggers, making it easier to manage and filter logs from different parts of the application.
- Customization:
  - Poco: Allows for different channels, formatters, and configurations to be set per logger.
  - spdlog: Provides the ability to set different sinks, formatters, and configurations per logger.

## 4. Advanced `spdlog` features

4.1 Asynchronous Logging

`spdlog`` supports asynchronous logging to boost performance.
When logging asynchronously, log messages are pushed into a lock-free queue and are processed by a dedicated thread.

In [ ]:
%%executable async_logger

#include <spdlog/async.h>
#include <spdlog/sinks/stdout_color_sinks.h>

int main() {
    // Initialize the async logger
    spdlog::init_thread_pool(8192, 1);  // queue size 8192, 1 backing thread
    auto async_logger = spdlog::stdout_color_mt<spdlog::async_factory>("async_logger");
    
    async_logger->info("This is an async log message");
    
    return 0;
}


In [ ]:
!./async_logger

4.2 Rotating File Logging

With `spdlog``, you can create rotating log files.
Once a log file reaches a certain size, a new file is created.

In [ ]:
%%executable rotating_logger

#include <spdlog/sinks/rotating_file_sink.h>

int main() {
    auto rotating_logger = spdlog::rotating_logger_mt("rotating_logger", "mylogfile", 1024 * 1024 * 5, 3);  // 5MB rotating log
    rotating_logger->info("This will be logged to a rotating log file");
    
    return 0;
}


In [ ]:
./rotating_logger

4.3 Multi-threading

`spdlog` is thread-safe and supports multi-threaded logging out of the box.


In [ ]:
%%executable thread_logger

#include <thread>

int main() {
    auto logger = spdlog::stdout_color_mt("mt_logger");
    
    std::thread t1([&](){
        logger->info("Logged from thread 1");
    });
    
    std::thread t2([&](){
        logger->info("Logged from thread 2");
    });
    
    t1.join();
    t2.join();
    
    return 0;
}

In [ ]:
./thread_logger

## 5. Where to go from here

`spdlog` is a header-only library, so here are a few ways to adopt it:

- use it as a replacement for some of the logging in your existing project
- for some small module, consider using `spdlog` as a local logger, in addition to the main logger based on `poco`.
- start using it in some satelite project, and see how it goes.
